In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from collections import defaultdict

In [572]:
##Path of chromedriver
chromedriver="./data/chromedriver.exe"


url_list=pd.read_csv('data/URLs.csv')

url_list

row=2
cricinfo_match_notes_url=(url_list.iloc[row]['Cricinfo_URL'])
cricbuzz_match_url=(url_list.iloc[row]['Cricbuzz_URL'])

cricinfo_match_notes_url



'https://www.espncricinfo.com/series/19430/scorecard/1152847/england-vs-australia-2nd-test-icc-world-test-championship-2019-2021'

In [573]:
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(cricinfo_match_notes_url)
cricinfo_matchnotes_soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
cricinfo_matchnotes_soup

<html class="js no-touch history cssanimations video localstorage desktop-ua scrollbars-17 desktop" lang="en" style=""><head><script async="" src="https://tracking.sokrati.com/javascripts/tracker.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-798217021" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script charset="UTF-8" src="https://cdn.taboola.com/libtrc/impl.20200211-14-RELEASE.js" type="text/javascript"></script><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script id="twitter-wjs" src="https://platform.twitter.com/widgets.js"></script><script async="" src="https://chuknu.sokrati.com/22623/tracker.js" type="text/javascript"></script><script src="https://a.espncdn.com/prod/scripts/prebid2.25.0.js" type="text/javascript"></script>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" ht

In [574]:
m_notes=cricinfo_matchnotes_soup.find('h1',text='Match Notes')
all_days=[d for d in m_notes.next_element.next.find_all('ul',{'class':'bulleted-list'})]
all_days.reverse()
all_days

[<ul class="bulleted-list" data-reactid="1380"><li data-reactid="1381">England 1st innings</li><li data-reactid="1382">Rain: England - 0/0</li><li data-reactid="1383">Lunch: England - 0/0</li><li data-reactid="1384">Rain: England - 0/0</li><li data-reactid="1385">Wet Ground: England - 0/0</li><li data-reactid="1386">Rain: England - 0/0</li><li data-reactid="1387">End Of Day: England - 0/0</li></ul>,
 <ul class="bulleted-list" data-reactid="1354"><li data-reactid="1355">Drinks: England - 31/2 in 11.5 overs (RJ Burns 14, JL Denly 2)</li><li data-reactid="1356">Over 15.6: Review by Australia (Bowling), Umpire - CB Gaffaney, Batsman - JL Denly (Struck down)</li><li data-reactid="1357">England: 50 runs in 17.5 overs (107 balls), Extras 1</li><li data-reactid="1358">3rd Wicket: 50 runs in 105 balls (RJ Burns 22, JL Denly 27, Ex 1)</li><li data-reactid="1359">Lunch: England - 76/2 in 27.0 overs (RJ Burns 34, JL Denly 27)</li><li data-reactid="1360">England: 100 runs in 33.4 overs (202 balls),

In [575]:

innings_list=[] 
innings_reviews=defaultdict(list)
innings_list.append([a.text for a in all_days[0] if "innings" in a.text and len(a.text)<=30])
##There are instances where cricinfo match notes has the word 'innings' which might not be due to an innings beginning

day_wise_reviews=[a.text for a in all_days[0] if a.text.startswith("Over") or "innings" in a.text]
##Append from day2 onwards to the day 1 list

if len(all_days)>=2:
    for ad in all_days[1:]:
        daylist=[a.text for a in ad if a.text.startswith("Over") or "innings" in a.text]
        innings_list.append([a.text for a in ad if "innings" in a.text and len(a.text)<=30])
        for d in daylist:
            day_wise_reviews.append(d)

idxs = [i for i,x in enumerate(day_wise_reviews) if 'innings' in x]
start_end_idxs=list(map(list, zip(idxs, idxs[1:])))
for s in start_end_idxs:
    innings_reviews[day_wise_reviews[s[0]]]=day_wise_reviews[s[0]+1:s[1]]
innings_reviews[day_wise_reviews[max(idxs)]]=day_wise_reviews[max(idxs)+1:]

##Make the innings_wise_reviews dictionary to dataframe

idf=pd.DataFrame.from_dict(innings_reviews,orient='index')
idf.reset_index(inplace=True)
idf.fillna('',inplace=True)
innings_df=idf.melt(id_vars='index',value_name='reviews')

##In this case, the variable column does not add any value. Hence it can be dropped
innings_df.drop(columns='variable',inplace=True) 
innings_df.columns=['innings','reviews']
innings_df=innings_df[innings_df.reviews!='']

###Augment Innings_df with the referral notes

over=[]
review_team=[]
review_umpire=[]
review_batsman=[]
review_outcome=[]


for review in innings_df.reviews:
    over.append(review.split('Over')[1].split(':')[0].strip())
    review_team.append(review.split(':')[1].strip().split('by ')[1].split(',')[0])
    review_umpire.append(review.split(':')[1].strip().split('Umpire - ')[1].split(',')[0])
    review_batsman.append(review.split(':')[1].strip().split('Batsman -')[1].strip().split('(')[0].strip())
    review_outcome.append(review.split(':')[1].strip().split('Batsman -')[1].strip().split('(')[1].split(')')[0].strip())

innings_df['Over']=over
innings_df['Review_team']=review_team
innings_df['Review_batsman']=review_batsman
innings_df['Review_umpire']=review_umpire
innings_df['Review_outcome']=review_outcome
innings_df['Umpires_call']=innings_df['Review_outcome'].apply(lambda x:"Umpire" in x)
innings_df['index']=range(len(innings_df.reviews))
innings_list_updated=[]
for i in innings_list:
    if(i):
        for a in range(len(i)):
            innings_list_updated.append(i[a])

##Remove duplicates if any, from innings list 

innings_list_updated=list(dict.fromkeys(innings_list_updated))
innings_list_updated.reverse()

innings_df.set_index('index',inplace=True,drop=False)
innings_df


,innings,reviews,Over,Review_team,Review_batsman,Review_umpire,Review_outcome,Umpires_call,index
index,,,,,,,,,
0,England 1st innings,"Over 15.6: Review by Australia (Bowling), Umpi...",15.6,Australia (Bowling),JL Denly,CB Gaffaney,Struck down,False,0
1,Australia 1st innings,"Over 22.5: Review by Australia (Batting), Umpi...",22.5,Australia (Batting),CT Bancroft,Aleem Dar,Struck down - Umpires Call,True,1
2,England 2nd innings,"Over 40.1: Review by Australia (Bowling), Umpi...",40.1,Australia (Bowling),BA Stokes,CB Gaffaney,Struck down,False,2
3,M Labuschagne replaced SPD Smith as a concussi...,"Over 57.5: Review by Australia (Bowling), Umpi...",57.5,Australia (Bowling),BA Stokes,Aleem Dar,Struck down,False,3
4,Australia 2nd innings,"Over 25.1: Review by England (Bowling), Umpire...",25.1,England (Bowling),M Labuschagne,Aleem Dar,Struck down,False,4
5,England 1st innings,"Over 57.5: Review by Australia (Bowling), Umpi...",57.5,Australia (Bowling),JM Bairstow,CB Gaffaney,Struck down,False,5
6,Australia 1st innings,"Over 30.1: Review by England (Bowling), Umpire...",30.1,England (Bowling),TM Head,Aleem Dar,Upheld,False,6
7,Australia 2nd innings,"Over 45.4: Review by England (Bowling), Umpire...",45.4,England (Bowling),PJ Cummins,Aleem Dar,Struck down,False,7
8,England 1st innings,"Over 64.6: Review by England (Batting), Umpire...",64.6,England (Batting),CR Woakes,Aleem Dar,Struck down,False,8


In [576]:
fow_text=[fow.text for fow in cricinfo_matchnotes_soup.find_all('div',{"class":"wrap dnb"}) if "Fall of wickets:" in fow.text]
innings_fow=defaultdict(list)

for a,inn in enumerate(reversed(innings_list_updated)):
    ##Handle retired
    innings_fow[inn]=[f.split(')')[0].strip().split(' ')[0] for i,f in enumerate(fow_text[a].split(':')[1].strip().split(',')) if i%2!=0 and f.split(')')[0].strip().split(' ')[0]!='retired']

innings_fow_df=pd.DataFrame.from_dict(innings_fow,orient='index')
innings_fow_df.reset_index(inplace=True)
innings_fow_df.fillna('',inplace=True)
innings_fow_df=innings_fow_df.melt(id_vars='index',value_name='wickets')
##In this case, the variable refers to the fall of wicket. 

innings_fow_df['variable']=innings_fow_df['variable']+1
innings_fow_df.columns=['innings','active_partnership','Over']
innings_fow_df=innings_fow_df[innings_fow_df.Over!='']

pbreak_innings=pd.merge(innings_df,innings_fow_df,on=['innings','Over'],how='inner')['index']
innings_df['Partnership_broken']=False
innings_df.loc[pbreak_innings,'Partnership_broken']=True
innings_df,innings_fow_df

(                                                 innings  \
 index                                                      
 0                                    England 1st innings   
 1                                  Australia 1st innings   
 2                                    England 2nd innings   
 3      M Labuschagne replaced SPD Smith as a concussi...   
 4                                  Australia 2nd innings   
 5                                    England 1st innings   
 6                                  Australia 1st innings   
 7                                  Australia 2nd innings   
 8                                    England 1st innings   
 9                                  Australia 1st innings   
 10                                 Australia 1st innings   
 11                                 Australia 1st innings   
 12                                 Australia 1st innings   
 
                                                  reviews  Over  \
 index          

In [578]:
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(cricbuzz_match_url)
cricbuzz_match_soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
cricbuzz_match_soup


<html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"/><script async="" src="https://sb.scorecardresearch.com/c2/6036484/cs.js" type="text/javascript"></script><script async="" src="https://www.googletagservices.com/tag/js/gpt.js" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-PGNCT7"></script><script async="" src="/dist/js/cricbuzz.min.6556.js" type="text/javascript"></script><script>var is_mobile = /symbian|tizen|midp|uc(web|browser)|MSIE (5.0|6.0|7.0|8.0)|tablet/i.test(navigator.userAgent);	if(is_mobile && window.location.hostname != 

### Process cricbuzz commentary to map innings and Over

In [580]:

commentary_text=[c.text for c in cricbuzz_match_soup.find_all('p',{'class':'cb-col cb-col-90 cb-com-ln'})]
over_text=[o.text for o in cricbuzz_match_soup.find_all('span',{'cb-col cb-col-8 text-bold'})]
inngs_breaks=[index for index, value in enumerate(over_text) if value == '0.1']
# current_innings=[]
# current_commentary=[]
# current_over=[]
# end=0
##Shortest innings buffer #of balls between innings is one over
s_innings_buffer=2

if len(inngs_breaks)>=5:
    inngs_idx_to_remove=inngs_breaks.index([s for s,t in list(zip(inngs_breaks,inngs_breaks[1:])) if t-s<=6][0])
    inngs_breaks.pop(inngs_idx_to_remove)

cricbuzz_commentary_df=pd.DataFrame({'Over':over_text,'commentary':commentary_text})

start=0

for nib, ib in enumerate(inngs_breaks):
    cricbuzz_commentary_df.loc[start:ib,'innings']=innings_list_updated[nib]
    start=ib+1

def merge_commentary(text):
    return '###'.join(text)

cricbuzz_commentary_df['commentary']=(cricbuzz_commentary_df.groupby(['Over','innings'],as_index=False)['commentary'].transform(merge_commentary).reset_index(drop=True))

cricbuzz_commentary_df.Over=cricbuzz_commentary_df.Over.astype('float')

cricbuzz_commentary_df.drop_duplicates(inplace=True)


# cricbuzz_commentary_df=cricbuzz_commentary_df.groupby(['innings','Over','commentary']).agg({'innings':'first','Over':'first','commentary':merge_commentary}).reset_index(drop=True)
#     start=b
#     for on,o in enumerate(over_text[end:start+1]):
#         current_innings.append(innings_list_updated[i])
#         current_over.append(o)
#         current_commentary.append(commentary_text[end:start+1][on])
#     end=start+1


In [581]:
innings_fow_df.Over=innings_fow_df.Over.astype('float')
cdf=cricbuzz_commentary_df.groupby('innings')['Over'].max()
idf=innings_fow_df.groupby('innings')['Over'].max()
idf2=pd.DataFrame(cdf.loc[cdf!=idf]).reset_index()

In [592]:
idf2.shape[0]>=1

True

In [590]:
if idf2.shape[0]>=1:
    idf_wicket=innings_fow_df.groupby('innings')['active_partnership'].max()
    idf_wicket=pd.DataFrame(idf_wicket[idf_wicket<=9])
    fow_df_to_add=idf2.merge(idf_wicket,on=['innings'])
    fow_df_to_add['active_partnership']=fow_df_to_add['active_partnership']+1


In [584]:
innings_fow_df=pd.concat([innings_fow_df,fow_df_to_add],axis=0)

C:\Users\k.shridhar\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [585]:
innings_fow_df.reset_index(inplace=True,drop=True)

innings_fow_df

,Over,active_partnership,innings
0,1.3,1,England 1st innings
1,4.2,1,Australia 1st innings
2,4.1,1,England 2nd innings
3,3.3,1,Australia 2nd innings
4,9.1,2,England 1st innings
5,22.5,2,Australia 1st innings
6,4.2,2,England 2nd innings
7,5.3,2,Australia 2nd innings
8,31.6,3,England 1st innings
9,23.2,3,Australia 1st innings


### Compile all tables

In [9]:
##SNIPPET TO EXTRACT ALL WTC SCORECARD URLS OF CRICINFO

# page_url='https://www.espncricinfo.com/scores/series/19430/season/2019/icc-world-test-championship'
# r1=requests.get(page_url)
# bs_main=BeautifulSoup(r1.text,'html.parser')
# urllist=[]
# for link in bs_main.find_all('a',href=True,text='SCORECARD'):
#     urllist.append('https://www.espncricinfo.com'+link['href'])
# pd.DataFrame(urllist).to_csv('url.csv')


In [586]:
innings_state=pd.merge(cricbuzz_commentary_df,innings_fow_df,on=['Over','innings'],how='left')
#last_fow_idx=min(innings_state[~pd.isnull(innings_state.active_partnership)].index)
# ##Process FOW for innings which were declared or not all out or won
# if last_fow_idx>=1:
#     last_fow_val=(innings_state[~pd.isnull(innings_state.active_partnership)]['active_partnership'])[last_fow_idx]+1
#     innings_state.loc[0:last_fow_idx-1]['active_partnership']=last_fow_val
# reviews_match=pd.merge(innings_state,innings_df,how='left')
# reviews_match.fillna('',inplace=True)
# reviews_match['match']=cricbuzz_match_url.split('/')[-1]
# reviews_match
    
# reviews_match.to_csv('data/reviews_match_{0}.csv'.format(cricbuzz_match_url.split('/')[-1]),index=False) 

In [587]:
is_active=innings_state.groupby(['innings']).ffill()

In [588]:
innings_state['active_partnership']=is_active['active_partnership']

In [600]:


reviews_match=pd.merge(innings_state,innings_df,how='left')

reviews_match

,Over,commentary,innings,active_partnership,reviews,Review_team,Review_batsman,Review_umpire,Review_outcome,Umpires_call,index,Partnership_broken
0,47.3,"Jack Leach to Pat Cummins, no run, slides in f...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47.2,"Jack Leach to Pat Cummins, no run, drifting in...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,47.1,"Jack Leach to Pat Cummins, no run, smothered b...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46.6,"Denly to Head, no run, smoothly driven by Head...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46.5,"Denly to Head, no run, defended past silly point",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46.4,"Denly to Head, no run, overpitched outside off...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46.3,"Denly to Head, no run, whenever there's a chan...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,46.2,"Denly to Head, no run, almost shaves the outsi...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,46.1,"Denly to Head, no run, overpitched outside off...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,45.6,"Jack Leach to Pat Cummins, no run, back round ...",Australia 2nd innings,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [589]:
innings_state.to_csv('innings.csv',index=False)

### Read match wise Cricinfo and Cricbuzz URLs

In [11]:
url_list=pd.read_csv('review_data/URLs.csv')

url_list

FileNotFoundError: [Errno 2] File b'review_data/URLs.csv' does not exist: b'review_data/URLs.csv'

### Debug

### Collect all match wise referral DF into one

In [ ]:
reviews_df=reviews_match

### Process review events- to add overturned, review loss, 1st review loss, 2nd review loss logics

In [ ]:
df=reviews_df.loc[(~pd.isnull(reviews_df.reviews))]

# df=df[df.match=='eng-vs-aus-3rd-test-the-ashes-2019']
# ##Split into innings
# reviews_df['Review_team']=reviews_df['Review_team'].astype('str')
# df=reviews_df
# df.fillna('',inplace=True)
##Keep start review count =2
## Overturn event logic. bowling-upheld and p.broken=True [OR] batting-upheld with p.broken=False 
##1st review lost at =
##2nd review lost at = 
## Review loss logic is take innings, take team batting- check upheld or struck down or struck down umpires call. add 1 if struck down
## for team bowling- check upheld or struck down or stuck down umpires call. if struck down, then add 1.

In [ ]:
df['overturned']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==True) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Upheld' and x.Partnership_broken==False)) else False,axis=1)

In [ ]:
df['review_lost']=df[['Review_team','Review_outcome','Partnership_broken']].apply(lambda x: True if 
(('Bowling' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==False) 
 or ('Batting' in x.Review_team and x.Review_outcome=='Struck down' and x.Partnership_broken==True)) else False,axis=1)


In [ ]:
df['first_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==1)

df['second_review_lost']=(df.loc[df.review_lost==True].sort_values('Over').groupby(["match","innings","innings_action"])['review_lost'].rank('first')==2)


In [ ]:
df.fillna('',inplace=True)

In [ ]:
cols_to_add_from_df=['overturned','review_lost','first_review_lost','second_review_lost']

for col in cols_to_add_from_df:
    reviews_df[col]=df.loc[df.index,col]

In [ ]:
reviews_df.fillna('',inplace=True)

In [ ]:
# reviews_df.to_csv('review_data/reviews_updated.csv')

In [ ]:
# reviews_df=pd.read_csv('review_data/reviews_updated.csv')

reviews_df.shape

### Visualisations for blog

- Batsman who took the most reviews (intiated by batsman)
- Whom did teams take a review against the most? 
- Umpire effectiveness (# of overturned decisions/total number of DRS calls for an umpire) Bar chart
- Review effectiveness by teams overall (Who had most partnership breaks when overturned)

In [ ]:
df=reviews_df[~pd.isnull(reviews_df.reviews)]

df.shape

print(df.columns)

### Overturned decisions per match

In [ ]:
overturned_per_match=df.groupby('match')['overturned'].sum()

### Total decisions per match

In [ ]:
total_per_match=df.groupby('match')['overturned'].count()

In [ ]:
ot_decisions=[]

tot_avg_decisions=[]

In [ ]:
for r in range((df.shape[0])):
    ot_decisions.append(overturned_per_match.loc[df.match.iloc[r]])
    tot_avg_decisions.append(overturned_per_match.loc[df.match.iloc[r]]/total_per_match.loc[df.match.iloc[r]])

In [ ]:
df['overturned_decisions']=ot_decisions

In [ ]:
df['ratio_overturned']=tot_avg_decisions

In [ ]:
# fig = px.bar(df,x='match',y='overturned_decisions')
# # fig.update_xaxes(categoryorder='sum descending')

# fig.show()

In [ ]:
df['team']=df['Review_team'].apply(lambda x:x.split('(')[0].strip())

df['bowler']=df['Commentary'].apply(lambda x:x.split('to')[0].strip())

In [ ]:
df

In [ ]:
df.shape

In [ ]:
# df.to_csv('review_data/reviews_15dec.csv',index=False)

In [ ]:
df1=df.loc[df.Review_team!='']

df1

In [ ]:
# df1.to_csv("review_data/rev_events.csv",index=False)

#### Fixes and enhancements:

- Active partnership sometimes increments additional fields- due to retired not outs and 1st innings declarations..etc.
- Add wicket keeper at review event